In [1]:
import numpy as np
import math as mt
from itertools import combinations

In [2]:
def RotAngleLig(theta):
    matrix = np.array(
    [[-np.cos(theta), -np.sin(theta), 0, 0],
    [  np.sin(theta), -np.cos(theta), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [3]:
def RotAngleTor(beta):
    matrix = np.array(
    [[1, 0, 0, 0],
    [ 0, np.cos(beta), -np.sin(beta),  0],
    [ 0, np.sin(beta),  np.cos(beta), 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [4]:
def Translacao(d):
    matrix = np.array([
    [1, 0, 0, d],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [5]:
def comb(theta, beta, d):
    return  RotAngleTor(beta)@RotAngleLig(theta)@Translacao(d)

In [6]:
print(comb(mt.pi, mt.pi/2, 0.2))

[[ 1.00000000e+00 -1.22464680e-16  0.00000000e+00  2.00000000e-01]
 [ 7.49879891e-33  6.12323400e-17 -1.00000000e+00  1.49975978e-33]
 [ 1.22464680e-16  1.00000000e+00  6.12323400e-17  2.44929360e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [71]:
def Positions(theta, beta, d):
    n = len(d)
   
    e4 = np.array([0, 0, 0, 1])
    
    positions = [np.zeros(3)]

    Id = np.eye(4)
    
    for i in range(n):
        Bi = comb(theta[i], beta[i], d[i])
        Id = Id @ Bi
        xi = Id @ e4
        positions.append(xi[:3])  
    return np.array(positions)

In [72]:
def Distances(positions):
    n = len(positions)


    distance = {}

    for i, j in combinations(range(n), 2):
        rij = np.linalg.norm(positions[j] - positions[i])
        distance[(i+1, j+1)] = rij
    return distance

In [73]:
def C_matrix(theta, omega, d):
    A = np.array([
        [-np.cos(theta), -np.sin(theta), 0],
        [ np.sin(theta)*np.cos(omega), -np.cos(theta)*np.cos(omega), -np.sin(omega)],
        [ np.sin(theta)*np.sin(omega), -np.cos(theta)*np.sin(omega),  np.cos(omega)]
    ])
    
    b = np.array([[-d*np.cos(theta)],
                  [ d*np.sin(theta)*np.cos(omega)],
                  [ d*np.sin(theta)*np.sin(omega)]])
   
    normb2 = float(np.sum(b**2))
    
   
    U = np.block([
        [A, b, np.zeros((3,1))],
        [np.zeros((1,3)), np.array([[1,0]])],
        [b.T @ A, np.array([[normb2/2, 1]])]
    ])
    return U


In [126]:
def conformalPositions(theta, omega, d):
    e0 = np.array([[0],[0],[0],[1],[0]])
    e_inf = np.array([[0],[0],[0],[0],[1]])

    
    B = [C_matrix(theta[i], omega[i], d[i]) for i in range(len(d))]

   
    Bcum = [np.eye(5)]
    for i in range(len(d)):
        Bi = C_matrix(theta[i], omega[i], d[i])
        Bcum.append(Bcum[-1] @ Bi)
        B.append(Bi)

    x_hat = [Bcum[i] @ e0 for i in range(len(d)+1)]

    
    x = [xh[:3].flatten() for xh in x_hat]

    
    x1 = Positions(theta, omega, d)
    distance_eucli = Distances(x1)
    



    return B, x_hat, x


In [141]:
def ConformalDistances(i, j, B):
    e0 = np.array([[0],[0],[0],[1],[0]])
    e_inf = np.array([[0],[0],[0],[0],[1]])

    if i >= len(B):
        return 0

    Bcum = [np.eye(5)]
    for Bi in B:
        Bcum.append(Bcum[-1] @ Bi)

    if j >= len(Bcum):
        j = len(Bcum) - 1

    # transformação entre os pontos i e j
    Bij = np.linalg.inv(Bcum[i]) @ Bcum[j]
    val = 2 * (e_inf.T @ Bij @ e0)
    return float(val)


In [142]:
#coluna 3: d
#coluna 5: theta
#coluna 7: omega 

In [143]:
#Vamos ler o arquivo para podermos aplicar o metodo

def LerMat(arquivo):
    atoms = []
    variables = {}

    with open(arquivo, 'r') as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]

    split_index = None
    for i, l in enumerate(lines):
        if any(char.isdigit() for char in l.split()[0]):
            split_index = i
            break

    if split_index is None:
        split_index = len(lines)

    for line in lines[:split_index]:
        parts = line.split()
        n = len(parts)
        if n in [1, 3, 5, 7]:
            atoms.append(parts)
        else:
            print("Aviso: linha malformada:", parts)

    for line in lines[split_index:]:
        parts = line.split()
        if len(parts) == 2:
            name, value = parts
            try:
                variables[name] = float(value)
            except ValueError:
                pass

    return atoms, variables

# Uso
atoms = LerMat('/home/rnatieli/Downloads/we02.zmat')[0]
values = LerMat('/home/rnatieli/Downloads/we02.zmat')[1]


In [144]:
atoms, values

([['O'],
  ['C', '1', 'dist02'],
  ['C', '2', 'dist03', '1', 'angl03'],
  ['C', '3', 'dist04', '2', 'angl04', '1', 'ptor04'],
  ['O', '2', 'dist05', '1', 'angl05', '3', 'itor05'],
  ['N', '3', 'dist06', '2', 'angl06', '4', 'itor06'],
  ['H', '4', 'dist07', '3', 'angl07', '2', 'ptor07'],
  ['H', '5', 'dist08', '2', 'angl08', '1', 'ptor08'],
  ['H', '6', 'dist09', '3', 'angl09', '2', 'ptor09'],
  ['H', '4', 'dist10', '3', 'angl10', '7', 'itor10'],
  ['H', '4', 'dist11', '3', 'angl11', '7', 'itor11'],
  ['H', '3', 'dist12', '2', 'angl12', '4', 'itor12'],
  ['H', '6', 'dist13', '3', 'angl13', '9', 'itor13']],
 {'dist02': 1.25669,
  'dist03': 1.46513,
  'angl03': 119.17335,
  'dist04': 1.5089,
  'angl04': 113.1617,
  'ptor04': 125.6476,
  'dist05': 1.38725,
  'angl05': 116.91927,
  'itor05': 179.99916,
  'dist06': 1.43372,
  'angl06': 110.90762,
  'itor06': -125.65195,
  'dist07': 1.11313,
  'angl07': 111.66282,
  'ptor07': 13.15326,
  'dist08': 1.01115,
  'angl08': 117.10283,
  'ptor08': -

In [145]:
def conversao(atoms, values):
   
    d, theta, omega = [], [], []

    #Converter os angulos para rad
    for entry in atoms[1:]:  
        # distancia
        if len(entry) >= 3:
            d.append(values.get(entry[2], 1.0))
        else:
            d.append(0.0)

        # ligacao
        if len(entry) >= 5:
            theta.append(np.deg2rad(values.get(entry[4], 90.0)))  
        else:
            theta.append(np.deg2rad(90.0))

        # torcao
        if len(entry) >= 7:
            omega.append(np.deg2rad(values.get(entry[6], 0.0)))  
        else:
            omega.append(0.0)
    return np.array(theta), np.array(omega), np.array(d)


In [150]:
def teste(theta, beta, d):
    e0 = np.array([[0],[0],[0],[1],[0]])
    e_inf = np.array([[0],[0],[0],[0],[1]])
    pos_h = Positions(theta, beta, d)

    
    B, c, xhat = conformalPositions(theta, beta, d)

    #Tem que normalizar por isso o loop
    c_cart = []
    for p in c:
        p = np.array(p).flatten()        
        denom = p[3]                     
        c_cart.append(p[:3] / denom) 
    c_cart = np.array(c_cart)    
    print("Par      Hom        Conf(cart)   Conf(teórica)")
    for i, j in combinations(range(len(c_cart)), 2):
        hom  = np.linalg.norm(pos_h[j] - pos_h[i])
        cart = np.linalg.norm(c_cart[j] - c_cart[i])
        conf = np.sqrt(abs(ConformalDistances(i, j, B)))
        print(f"({i+1},{j+1})  {hom:}   {cart:}   {conf:}")


In [151]:
theta, omega, d = conversao(atoms, values)

teste(theta, omega, d)

Par      Hom        Conf(cart)   Conf(teórica)
(1,2)  1.25669   1.25669   1.25669
(1,3)  2.349657126755717   2.349657126755717   2.349657126755717
(1,4)  3.4696081511972845   3.4696081511972845   3.469608151197285
(1,5)  4.661170306814135   4.661170306814135   4.661170306814135
(1,6)  5.734980802287568   5.734980802287568   5.734980802287568
(1,7)  5.556358369044915   5.556358369044915   5.556358369044915
(1,8)  4.989198707182842   4.989198707182842   4.989198707182843
(1,9)  5.542324349534713   5.542324349534713   5.542324349534714
(1,10)  6.075823182698313   6.075823182698314   6.075823182698314
(1,11)  7.166961818701707   7.166961818701707   7.166961818701708
(1,12)  7.642242175820707   7.642242175820707   7.642242175820709
(1,13)  8.422006141949879   8.422006141949879   8.422006141949883
(2,3)  1.46513   1.46513   1.4651300000000003
(2,4)  2.482432320213711   2.482432320213711   2.4824323202137113
(2,5)  3.746804641134222   3.746804641134222   3.746804641134222
(2,6)  4.65395372180

/tmp/ipykernel_22787/1058074376.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(val)
